In [1]:
import pandas as pd
import os
import numpy as np
from ecgdetectors import Detectors
from time import time as time
import seaborn as sns
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values, get_time_domain_features, get_geometrical_features, get_frequency_domain_features, get_csi_cvi_features, get_poincare_plot_features, get_sampen
from time import time as time
import warnings
warnings.filterwarnings('ignore')

In [2]:
fs = 250

In [8]:
df = pd.read_csv('finaldfs/2017final_normal_rpeak_L5_S1.csv')
df.reset_index(drop = True, inplace = True)

In [9]:
df

,af,0,1,2,3,4,5,6,7,8,...,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163588,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163589,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163590,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163591,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
%%time
def featureExtraction(df, fs):
    df1 = df.af.copy()
    df.drop(columns = ['af'], inplace = True)
    final = list()
    for i in range(len(df)):
        peaks = []
        peaks = df.loc[i][df.loc[i] == 1].index.values
        peaks = [int(x) for x in peaks]
        if (len(peaks) == 0 or len(peaks) == 1):
            continue
        af = df1.loc[i]
        rr = np.diff(peaks) / fs 
        hr = 60 / rr
        rmssd = np.sqrt(np.mean(np.square(rr)))
        sdnn = np.std(rr)
        mean_rr = np.mean(rr)
        mean_hr = np.mean(hr)
        std_hr = np.std(hr)
        min_hr = np.min(hr)
        max_hr = np.max(hr)
        single_window_df = pd.DataFrame([[rmssd, sdnn, mean_rr, mean_hr, std_hr, min_hr, max_hr]], columns = ['RMSSD', 'STDNN', 'MEAN_RR', 'MEAN_HR', 'STD_HR', 'MIN_HR', 'MAX_HR'])
        single_window_df.insert(0, 'af', af)
        final.append(single_window_df)
        del single_window_df
    return pd.concat(final)

finaldf = featureExtraction(df, fs)
finaldf.reset_index(drop = True, inplace = True)

CPU times: user 8min 44s, sys: 1.72 s, total: 8min 46s
Wall time: 8min 46s


In [11]:
finaldf1 = finaldf.copy()

In [14]:
finaldf1

,af,RMSSD,STDNN,MEAN_RR,MEAN_HR,STD_HR,MIN_HR,MAX_HR
0,0,0.763275,0.036538,0.7624,78.881404,3.810557,74.257426,83.798883
1,0,0.784564,0.029755,0.7840,76.646108,3.044678,73.891626,82.872928
2,0,0.799334,0.014621,0.7992,75.100714,1.401583,73.891626,77.720207
3,0,0.793752,0.015513,0.7936,75.633844,1.483975,73.891626,77.720207
4,0,0.800080,0.011314,0.8000,75.015176,1.073150,73.891626,76.923077
...,...,...,...,...,...,...,...,...
162623,0,0.922219,0.020100,0.9220,65.107552,1.451010,63.829787,67.567568
162624,0,0.923231,0.020664,0.9230,65.038773,1.489974,63.829787,67.567568
162625,0,0.924251,0.021541,0.9240,64.971157,1.548300,63.559322,67.567568
162626,0,0.933032,0.007681,0.9330,64.313031,0.528290,63.559322,64.935065


In [13]:
finaldf1.af.value_counts()

0    141350
1     21278
Name: af, dtype: int64

In [64]:
finaldf1 = finaldf1.drop(finaldf1[finaldf1['af'].eq(0)].sample(10000).index)

In [65]:
finaldf1.reset_index(drop = True, inplace = True)

In [16]:
finaldf1.to_csv('/hdd/physio/af2/finaldfs/2017final_normal_feats_L5_S1.csv')